Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [57]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [67]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [68]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [69]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [70]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  b1 = tf.Variable(tf.zeros([hidden_size]))

  # Variables.
  w2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  net = lambda tf_data: tf.matmul(tf.nn.relu(tf.matmul(tf_data, w1) + b1), w2) + b2
  logits = net(tf_train_dataset)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
  )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(net(tf_train_dataset))
  valid_prediction = tf.nn.softmax(net(tf_valid_dataset))
  test_prediction = tf.nn.softmax(net(tf_test_dataset))

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch offset at step %d: %d" % (offset, batch_size))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 402.179199
Minibatch offset at step 0: 128
Minibatch accuracy: 7.8%
Validation accuracy: 33.8%
Minibatch loss at step 500: 17.493740
Minibatch offset at step 64000: 128
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 1000: 7.914797
Minibatch offset at step 128000: 128
Minibatch accuracy: 82.0%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 4.945500
Minibatch offset at step 192000: 128
Minibatch accuracy: 87.5%
Validation accuracy: 78.7%
Minibatch loss at step 2000: 2.505423
Minibatch offset at step 56128: 128
Minibatch accuracy: 88.3%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 2.899067
Minibatch offset at step 120128: 128
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Minibatch loss at step 3000: 1.606115
Minibatch offset at step 184128: 128
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Test accuracy: 89.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
batch_size = 1024
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  b1 = tf.Variable(tf.zeros([hidden_size]))

  # Variables.
  w2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  net = lambda tf_data: tf.matmul(tf.nn.relu(tf.matmul(tf_data, w1) + b1), w2) + b2
  logits = net(tf_train_dataset)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
      + 0.01*tf.nn.l2_loss(w1)
      + 0.01*tf.nn.l2_loss(w2)
 )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(net(tf_train_dataset))
  valid_prediction = tf.nn.softmax(net(tf_valid_dataset))
  test_prediction = tf.nn.softmax(net(tf_test_dataset))

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch offset at step %d: %d" % (offset, batch_size))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3434.936279
Minibatch offset at step 0: 1024
Minibatch accuracy: 14.5%
Validation accuracy: 41.2%
Minibatch loss at step 500: 21.285080
Minibatch offset at step 114048: 1024
Minibatch accuracy: 84.4%
Validation accuracy: 85.3%
Minibatch loss at step 1000: 0.861748
Minibatch offset at step 29120: 1024
Minibatch accuracy: 84.9%
Validation accuracy: 84.3%
Minibatch loss at step 1500: 0.718329
Minibatch offset at step 143168: 1024
Minibatch accuracy: 83.8%
Validation accuracy: 84.3%
Minibatch loss at step 2000: 0.722449
Minibatch offset at step 58240: 1024
Minibatch accuracy: 83.1%
Validation accuracy: 84.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  b1 = tf.Variable(tf.zeros([hidden_size]))

  tf_keep_prob = tf.placeholder(tf.float32)

  # Variables.
  w2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  net = lambda tf_data: tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_data, w1) + b1), tf_keep_prob), w2) + b2
  logits = net(tf_train_dataset)

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
      + 0.01*tf.nn.l2_loss(w1)
      + 0.01*tf.nn.l2_loss(w2)
  )

  #accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(tf_train_labels, 1)), "float"))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(net(tf_train_dataset))
  valid_prediction = tf.nn.softmax(net(tf_valid_dataset))
  test_prediction = tf.nn.softmax(net(tf_test_dataset))

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, tf_keep_prob : 0.5}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch offset at step %d: %d" % (offset, batch_size))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval({tf_keep_prob : 0.5}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval({tf_keep_prob : 0.5}), test_labels))

In [ ]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_test_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))

  tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_test_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

  
  # Variables.
  w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  b1 = tf.Variable(tf.zeros([hidden_size]))

  tf_keep_prob = tf.placeholder(tf.float32)

  # Variables.
  w2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_dataset, w1) + b1), tf_keep_prob), w2) + b2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_labels)
      + 0.01*tf.nn.l2_loss(w1)
      + 0.01*tf.nn.l2_loss(w2)
  )

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  model = tf.nn.softmax(logits)

  performance = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(tf_test_labels, 1)), "float"))


num_steps = 31

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    train_dict = {tf_dataset : batch_data, tf_labels : batch_labels, tf_keep_prob : 0.5}
    valid_dict = {tf_test_dataset : valid_dataset, tf_test_labels : valid_labels, tf_keep_prob : 0.5}
    test_dict  = {tf_test_dataset : test_dataset, tf_test_labels : test_labels, tf_keep_prob : 0.5}

    _, l, predictions = session.run([optimizer, loss, model], feed_dict=train_dict)
    

    if (step % 5 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch offset at step %d: %d" % (offset, batch_size))
       print("Minibatch accuracy: %.1f%%" % performance.eval(train_dict))
      print("Validation accuracy: %.1f%%" % performance.eval(valid_dict))
  print("Test accuracy: %.1f%%" % performance.eval(test_dict))

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
